# Preparing the work_experiences.csv file

In [23]:
import pandas as pd
import os

In [24]:
path = os.path.join('OriginalData', 'work_experiences.csv')
data = pd.read_csv(path)

In [25]:
data

,user_id,company_id,location,start_year_month
0,53442,2651,"Istanbul, Turkey",201509
1,34558,815,"Istanbul, Turkey",201210
2,63761,26354,NaN,200010
3,10738,89,NaN,201610
4,8711,3113,"Istanbul, Turkey",201801
...,...,...,...,...
187014,22180,15065,Gebze,201205
187015,55822,25076,stajer,201506
187016,13750,1607,NaN,201901
187017,3679,1414,"Ankara, Turkey",201807


In [26]:
# Show summary of NaN values
data.isnull().sum()

user_id                 0
company_id              0
location            61414
start_year_month        0
dtype: int64

In [27]:
# Show how many unique values there are for each column
data.nunique()

user_id             57079
company_id          20837
location             6127
start_year_month      397
dtype: int64

In [28]:
# Assume NaN values are Turkey
data.fillna('Turkey', inplace=True)

In [29]:
# Put a '-' at in between the year and day in the format "yyyydd"
data['start_year_month'] = data['start_year_month'].apply(lambda x: str(x)[:4] + '-' + str(x)[4:])
# Turn the start_year_month column into a datetime object
data['start_year_month'] = pd.to_datetime(data['start_year_month'], format='%Y-%m')
# Rename the column to "start_date"
data.rename(columns={'start_year_month': 'start_date'}, inplace=True)

In [8]:
# Drop rows where the start_date is after 2019, using datetime
data.drop(data[data['start_date'] > '2019-01-01'].index, inplace=True)

In [32]:
data[data['start_date'] > '2019-01-01']

,user_id,company_id,location,start_date
12,20753,35,"Istanbul, Turkey",2019-10-01
15,12816,10649,Turkey,2019-05-01
22,48615,194,Turkey,2019-08-01
30,16486,3581,"Istanbul, Turkey",2019-06-01
36,27136,955,İstanbul,2019-08-01
...,...,...,...,...
186998,64402,1495,"Kocaeli, Turkey",2019-10-01
187003,24364,1562,Turkey,2019-07-01
187006,24832,16069,"Istanbul, Turkey",2019-07-01
187010,3356,2155,"Izmir, Turkey",2019-07-01


In [9]:
# Create a new column called "companies_changed" for each user_id
# This column will be the amount of companies the user has changed
data['companies_worked'] = data.groupby('user_id')['company_id'].transform('nunique')

The location column is all over the place, so we normalize it.

In [10]:
# Make country column lowercase
data['location'] = data['location'].apply(lambda x: str(x).lower())

# If country contains Turkish characters ş ü ö or ğ, change it to "Turkey"
data['country'] = data['location'].apply(lambda x: 'TR' if 'ş' in str(x) 
                                        or 'ü' in str(x) or 'ö' in str(x)
                                        or 'ğ' in str(x) or 'ı' in str(x)
                                        or 'ç' in str(x) or 'turkey' in str(x) 
                                        or 'türkiye' in str(x) or 'i̇' in str(x)
                                        else 'Other')

# If a user has both TR and Other, change it to MIX
data['country'] = data.groupby('user_id')['country'].transform(lambda x: 'MIX' if 'TR' in x.values and 'Other' in x.values else x)


In [11]:
# Create a new column called "max_duration" for each user_id
# This is the maximum difference between the start dates of the user's work experiences
data['max_duration'] = data.groupby('user_id')['start_date'].transform('max') - data.groupby('user_id')['start_date'].transform('min')

In [12]:
# For rows where the max_duration is 0, calculate the time between it's start_date and today
data.loc[data['max_duration'] == '0 days', 'max_duration'] = pd.to_datetime('2019-01-01') - data.loc[data['max_duration'] == '0 days', 'start_date']


In [13]:
# Show user 53442
data[data['user_id'] == 53442]

,user_id,company_id,location,start_date,companies_worked,country,max_duration
0,53442,2651,"istanbul, turkey",2015-09-01,3,TR,1918 days
23603,53442,4957,"kocaeli, türkiye",2012-11-01,3,TR,1918 days
44058,53442,2651,"istanbul, turkey",2018-02-01,3,TR,1918 days
57313,53442,3803,turkey,2013-09-01,3,TR,1918 days


In [14]:
# Drop location column
data.drop('location', axis=1, inplace=True)
# Drop start_date column
data.drop('start_date', axis=1, inplace=True)

Now we only want a single row for each user_id, a summary cleaned up for each user

In [15]:
# Show user 53442
data[data['user_id'] == 53442]

,user_id,company_id,companies_worked,country,max_duration
0,53442,2651,3,TR,1918 days
23603,53442,4957,3,TR,1918 days
44058,53442,2651,3,TR,1918 days
57313,53442,3803,3,TR,1918 days


In [16]:
# Drop company_id column
data.drop('company_id', axis=1, inplace=True)
# Leave only the first row for each user_id
data.drop_duplicates(subset='user_id', keep='first', inplace=True)

In [17]:
# Show user 53442
data[data['user_id'] == 53018]

,user_id,companies_worked,country,max_duration
31252,53018,3,TR,1310 days


- Sort and convert days to int

In [18]:
data.sort_values(by='user_id', inplace=True)
data.max_duration = data.max_duration.apply(lambda x: x.days)

In [19]:
data

,user_id,companies_worked,country,max_duration
147720,0,1,MIX,4870
760,2,3,TR,547
30839,5,1,TR,579
17649,7,2,MIX,883
4573,10,2,MIX,1734
...,...,...,...,...
133051,66269,1,TR,5357
94856,66270,1,TR,1491
24772,66271,7,TR,5479
13280,66272,5,TR,2344


In [20]:
# Save data to a new csv file in PreparedData folder
data.to_csv(os.path.join('PreparedData', 'work_experiences.csv'), index=False)

## For the country column
TR means user only worked at TR  
MIX means user worked at TR and Other  
Other means user only worked abroad  